# **AI Storytelling**

<img src="assets/logo.jpg" width="500" height="300">

This is Natural Language Processing platform that targets on conversion of short stories to audiobooks with features:
- characters extraction from text,
- voice generation by dialogues, narration and characters,
- musical background creation by text,
- activities sound generation by lines, and
- combination of all above features to create the audiobook.

Based on the success of the project, it could be extended with image or video processing features in upcoming days.

### Import Libraries and Functions

In [1]:
# basic libraries
import os
import pandas as pd

In [2]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# nlp libraries
import spacy
nlp = spacy.load("en_core_web_lg")

In [4]:
# text-to-speech libraries
from gtts import gTTS
from moviepy.editor import concatenate_audioclips, AudioFileClip
import librosa
import soundfile as sf

### Files and Dataframes Defining

In [5]:
# checking stories
os.listdir("stories")

['beautiful garden.txt', 'cafe at midnight.txt', 'the painted door.txt']

In [6]:
# input title
title = input("Enter text file name:")

Enter text file name: cafe at midnight


In [7]:
# opening text file
story = f"stories/{title}.txt"
with open(story, "r") as f:
    text = f.read()

In [8]:
# create dataframes
df_characters = pd.DataFrame(columns=["cid", "name", "frequency", "features"])
df_lines = pd.DataFrame(columns=["pid", "ndid", "character", "dialogue", "narration"])

# 1. Lines Identification

In [9]:
# converting text to paragraphs
paragraphs = text.split("\n")
non_empty_paragraphs = list(filter(lambda x: x != '', paragraphs))

In [10]:
# function to identify narrations and dialogues
def identify_narrations_and_dialogues(paragraph):
    """
    :param paragraph: string of paragraph in a story
    :return: list of tuples in (id, name_of_speaker, dialogue, narration) format
    """
    divisions = paragraph.split('"')
    divisions = list(filter(lambda x: x != '', divisions))
    i, spoken = 0, False
    narrations_and_dialogues = []
    for division in divisions:
        start_index = paragraph.find(division)
        end_index = start_index + len(division) - 1
        try:
            if '"'==paragraph[start_index-1:start_index] and '"'==paragraph[end_index+1:end_index+2] and spoken==False:
                # dialogues
                narrations_and_dialogues.append((i, None, division, None))
                spoken = True
            else:
                # narrations
                narrations_and_dialogues.append((i, None, None, division))
                spoken = False
        except:
            # narrations
            narrations_and_dialogues.append((i, None, None, division))
            spoken = False
        i += 1
    return narrations_and_dialogues


In [11]:
# identifying lines (narrations or dialogues) from each paragraphs
pid_num = 0
for paragraph in non_empty_paragraphs:
    for row in [(pid_num,)+nad for nad in identify_narrations_and_dialogues(paragraph)]:
        df_lines = df_lines._append(pd.Series(row, index=df_lines.columns), ignore_index=True)
    pid_num += 1

In [12]:
# set index using paragraph and nd
df_lines.set_index(['pid', 'ndid'], inplace=True)

In [13]:
# narrations and dialogues
df_lines.head(10)

character                                           dialogue  \
pid ndid                                                                
0   0         None                                               None   
1   0         None                                               None   
2   0         None                                               None   
    1         None                 Late night for a coffee, isn't it?   
3   0         None                                               None   
    1         None         Some stories unfold when the world sleeps.   
4   0         None                                               None   
    1         None                              What kind of stories?   
5   0         None  Stories of the forgotten, the ones that only e...   
    1         None                                               None   

                                                  narration  
pid ndid                                                     
0   0     Midnight in the heart of the city, the neon si...  
1   0     The door creaked open, and a mysterious figure...  
2   0                              Alex raised an eyebrow.   
    1                                                  None  
3   0     The stranger chuckled, a voice tinged with int...  
    1                                                  None  
4   0                      Intrigued, Alex leaned forward.   
    1                                                  None  
5   0                                                  None  
    1                    the stranger replied mysteriously.

# 2. Characters Identification

In [14]:
# function to find subjects speaking corresponding dialogues
def identify_subjects(sentence):
    """
    :param sentence: string of sentence in the story
    :return: individual speaker
    """
    tokens = []
    individual, individuals = '', []
    
    for tok in nlp(sentence):
        tokens.append([tok, tok.dep_])
        
    for i in range(len(tokens)-1, -1, -1):
        if tokens[i][1] == "nsubj":
            individual = str(tokens[i][0])
        elif (tokens[i][1] in ['comp', 'det'] and individual != ''):
            individual = str(tokens[i][0]) + " " + individual
        else:
            individuals.append(individual)
            
    individuals.append(individual)
    return list(set([ind for ind in individuals if ind != '']))

In [15]:
# finding semantic similarity
def calculating_semantic_similarity(subject, sentence):
    """
    :param subject: list of strings available in a sentence
    :return: similarity score
    """
    subject_doc = nlp(subject)
    sentence_doc = nlp(sentence)
    
    subject_vec = subject_doc.vector
    sentence_vec = sentence_doc.vector
    
    subject_vec_norm = subject_doc.vector_norm
    sentence_vec_norm = sentence_doc.vector_norm
    
    if subject_vec_norm > 0 and sentence_vec_norm > 0:
        similarity_score = subject_vec.dot(sentence_vec) / (subject_vec_norm * sentence_vec_norm)
    else:
        similarity_score = 0
    
    return similarity_score

In [16]:
# finding subjects from dialogues and narrations
dlg_list = df_lines.index[df_lines['dialogue'].notna()].tolist()

for dlgi in dlg_list:
    if df_lines.loc[(dlgi[0], dlgi[1]), 'dialogue'][-1] == ',':
        subjects = identify_subjects(df_lines.loc[(dlgi[0], dlgi[1]+1), 'narration'])
        relevant_dictionary = dict((subject.lower(), calculating_semantic_similarity(subject.lower(), df_lines.loc[(dlgi[0], dlgi[1]+1), 'narration'].lower())) for subject in subjects)
        relevant_subject = max(relevant_dictionary, key=lambda k: relevant_dictionary[k])
    else:
        subjects = identify_subjects(df_lines.loc[(dlgi[0], dlgi[1]-1), 'narration'])
        relevant_dictionary = dict((subject.lower(), calculating_semantic_similarity(subject.lower(), df_lines.loc[(dlgi[0], dlgi[1]-1), 'narration'].lower())) for subject in subjects)
        relevant_subject = max(relevant_dictionary, key=lambda k: relevant_dictionary[k])
    df_lines.loc[(dlgi[0], dlgi[1]), 'character'] = relevant_subject

In [17]:
# dialogues and corresponding characters
df_lines.head(10)

character                                           dialogue  \
pid ndid                                                                    
0   0             None                                               None   
1   0             None                                               None   
2   0             None                                               None   
    1             alex                 Late night for a coffee, isn't it?   
3   0             None                                               None   
    1     the stranger         Some stories unfold when the world sleeps.   
4   0             None                                               None   
    1             alex                              What kind of stories?   
5   0     the stranger  Stories of the forgotten, the ones that only e...   
    1             None                                               None   

                                                  narration  
pid ndid                                                     
0   0     Midnight in the heart of the city, the neon si...  
1   0     The door creaked open, and a mysterious figure...  
2   0                              Alex raised an eyebrow.   
    1                                                  None  
3   0     The stranger chuckled, a voice tinged with int...  
    1                                                  None  
4   0                      Intrigued, Alex leaned forward.   
    1                                                  None  
5   0                                                  None  
    1                    the stranger replied mysteriously.

In [18]:
# gathering characters from df_lines
characters = df_lines['character'].value_counts()
df_characters['name'] = characters.index
df_characters['frequency'] = characters.values

In [19]:
# declaring cid as index
df_characters['cid'] = range(0, len(df_characters))
df_characters.set_index('cid', inplace=True)

In [20]:
# characters
df_characters.head(10)

,name,frequency,features
cid,,,
0,alex,4,NaN
1,the stranger,4,NaN


# 3. Audio Generation

In [21]:
# creating audios
print("Step 1: CONVERSIONS")
i = 0
for index, row in df_lines.iterrows():
    if row['dialogue'] is not None:
        speech_gtts = gTTS(text=row['dialogue'], lang='en', slow=False, tld='co.in')
    else:
        speech_gtts = gTTS(text=row['narration'], lang='en', slow=False, tld='ie')
    temp_file = f"conversions/{i}.mp3"
    print(f"Line {i+1}/{df_lines.shape[0]} converted.")
    speech_gtts.save(temp_file)
    i += 1
print("Conversions finished.")

Step 1: CONVERSIONS
Line 1/19 converted.
Line 2/19 converted.
Line 3/19 converted.
Line 4/19 converted.
Line 5/19 converted.
Line 6/19 converted.
Line 7/19 converted.
Line 8/19 converted.
Line 9/19 converted.
Line 10/19 converted.
Line 11/19 converted.
Line 12/19 converted.
Line 13/19 converted.
Line 14/19 converted.
Line 15/19 converted.
Line 16/19 converted.
Line 17/19 converted.
Line 18/19 converted.
Line 19/19 converted.
Conversions finished.


In [22]:
# combining audios
print("Step 2: COMBINATION")
clips = [AudioFileClip(f"conversions/{i}.mp3") for i in range(df_lines.shape[0])]
final_clip = concatenate_audioclips(clips)
print("Combination finished.")

Step 2: COMBINATION
Combination finished.


In [35]:
# adjust speed
print("Step 3: ADJUSTMENT")
final_clip.write_audiofile("conversions/final_slow.mp3")
print("final_slow.mp3 downloaded.")
y, sr = librosa.load("conversions/final_slow.mp3", sr=None)
y_speed = librosa.effects.time_stretch(y, rate=1.25)
print("Speeded over.")

Step 3: ADJUSTMENT
MoviePy - Writing audio in conversions/final_slow.mp3


MoviePy - Done.
final_slow.mp3 downloaded.
Speeded over.


In [36]:
# removing audios
print("Step 4: REMOVAL")
for i in range(df_lines.shape[0]):
    os.remove(f"conversions/{i}.mp3")
    print(f"Removed {i}.mp3")
os.remove("conversions/final_slow.mp3")
print("Removal over.")

Step 4: REMOVAL
Removed 0.mp3
Removed 1.mp3
Removed 2.mp3
Removed 3.mp3
Removed 4.mp3
Removed 5.mp3
Removed 6.mp3
Removed 7.mp3
Removed 8.mp3
Removed 9.mp3
Removed 10.mp3
Removed 11.mp3
Removed 12.mp3
Removed 13.mp3
Removed 14.mp3
Removed 15.mp3
Removed 16.mp3
Removed 17.mp3
Removed 18.mp3
Removal over.


In [37]:
# downloading final audio
print("Step 5: DOWNLOADING")
final_title = title.replace(" ", "_")
sf.write(f"audiobooks/{final_title}.mp3", y_speed, sr)
print(f"{final_title}.mp3 downloaded")

Step 5: DOWNLOADING
cafe_at_midnight.mp3 downloaded
